In [1]:
import logging

import pandas as pd
import numpy as np
from ast import literal_eval as load
import sklearn

from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


ModuleNotFoundError: No module named 'transformers.models'

In [12]:
df = pd.read_csv('../interim/try1.csv')
df = df.rename(columns={'idx': 'indx'})
df['msk']=1
print(len(df))
for i in range(len(df)):
    if len(load(df.iloc[i,3]))<2:
        df.iloc[i,16]=0
df = df[df['msk']==1]
print(len(df))
df.insert(loc=0, column='idx', value=np.arange(len(df)))
df = df.sample(frac=1, random_state=1)
bound = int(0.9*len(df))
t_df = df[:bound]
e_df = df[bound:]
t_df = t_df.values
e_df = e_df.values

6128
4075


In [13]:
def convert(arr):
    ls=[]
    for i in range(len(arr)):
        lth = len(load(arr[i,4]))
        for p1 in range(lth-1):
            for p2 in range(p1+1, lth):
                phrase1=load(arr[i,4])[p1]
                phrase2=load(arr[i,4])[p2]
                word_ls=arr[i,2].split(' ')
                word_ls.insert(phrase1[1][0], '[[')
                word_ls.insert(phrase1[1][1]+1, ']]')
                word_ls.insert(phrase2[1][0]+2, '[[')
                word_ls.insert(phrase2[1][1]+3, ']]')
                flg=0
                if arr[i,6]=='[]':
                    trip_ls=[]
                else:
                    trip_ls=load(arr[i,6])
                    for trip in trip_ls:
                        if trip[0]==phrase1[0] and trip[2]==phrase2[0]:
                            if trip[1]=='has':
                                flg=1
                                break
                            elif trip[1]=='name':
                                flg==2
                                break
                ls.append([phrase1[0],phrase2[0],trip_ls,' '.join(word_ls),flg])
    dataframe = pd.DataFrame(ls)
    dataframe.columns = ['phrase1', 'phrase2', 'triples', 'text', 'labels']
    return dataframe



In [14]:
train_df = convert(t_df)
eval_df = convert(e_df)
num_negative = len(train_df[train_df['labels'] == 0])
weight_list = [1]
for i in range(1,3):
    weight_list.append(num_negative/len(df[df['labels'] == i]))
train_df.to_csv('train_B.csv')
eval_df.to_csv('eval_B.csv')

In [ ]:
model_args = ClassificationArgs()
# arguments for early stop
model_args.use_early_stopping = True
# model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "F1_score"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 18
model_args.early_stopping_consider_epochs = True
# model_args.evaluate_during_training_steps = 500
model_args.evaluate_during_training_verbose = True

model_args.train_batch_size = 16
model_args.learning_rate = 1e-5
model_args.scheduler = "linear_schedule_with_warmup"
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.save_steps = -1
model_args.save_model_every_epoch = False
model_args.no_save = True
model_args.output_dir = 'outputsB/'
model_args.best_model_dir = 'outputsB/best_model'
model_args.fp16 = False
model_args.evaluate_during_training = True
model_args.manual_seed = 1
model_args.use_multiprocessing = True
model_args.num_train_epochs = 20
model_args.gradient_accumulation_steps = 4
model_args.do_lower_case = True  # when using uncased model

In [ ]:
def triple_F1(ref, pred):
    TP=FP=FN=0
    for i in range(len(pred)):
        ref_ls = eval_df.iloc[i,2]
        pred_ls = []
        if pred[i]!=0:
            if pred[i]==1:
                trip = [eval_df.iloc[i,0], 'has', eval_df.iloc[i,1]]
            elif pred[i]==2:
                trip = [eval_df.iloc[i,0], 'name', eval_df.iloc[i,1]]
            pred_ls.append(trip)
        TP+=len([t for t in pred_ls if t in ref_ls])
        FP+=len([t for t in pred_ls if t not in ref_ls]) 
        FN+=len([t for t in ref_ls if t not in pred_ls])
    FN+=missed
    pc=TP/(TP+FP)
    rc=TP/(TP+FN)
    F1=2*pc*rc/(pc+rc)
    print(f'precision {pc}, recall {rc}, F1 {F1}')
    return F1

In [ ]:
model = ClassificationModel(
    "bert",
    "allenai/scibert_scivocab_uncased",
    weight=weight_list,
    num_labels=3,
    args=model_args,
)
print(f'weight list: {weight_list}')
# Train the model
model.train_model(train_df, eval_df=eval_df,F1_score=triple_F1)
# Evaluate the model
#result, model_outputs, wrong_predictions = model.eval_model(eval_df, F1_score=triple_F1)